In [2]:
sc

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
from pyspark.sql.functions import col
import math
import random
import itertools

from models.als import predictions_ALS
from models.MF_SGD import matrix_factorization_SGD
from models.means import *
from models.medians import *
from helpers import *
from models.helpers_scipy import *
from rescaler import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = load_csv()
df.head()

,User,Movie,Rating
0,44,1,4
1,61,1,3
2,67,1,4
3,72,1,3
4,86,1,5


In [5]:
df.shape[0]

1176952

In [6]:
np.random.seed(42)

In [7]:
l = [1, 2, 3, 4, 5, 6, 7]
l[0:2]

[1, 2]

In [31]:
from cross_validator import *
cv2 = CrossValidator()
%reload_ext autoreload
%autoreload 2



a = cv2.shuffle_indices(df, 3)

b = cv2.cross_validate(df, global_mean, 'global_mean')

global_mean


In [32]:
cv2.evaluate_model('global_mean', df)

1.1180021257962498

In [33]:
cv2.print_models()

['global_mean']


In [80]:
pred_dict = cv2.load_model(['global_mean'])

In [85]:
pred_dict['global_mean'][2]

,User,Movie,Rating
0,44,1,3.85677
1,67,1,3.85677
2,72,1,3.85677
3,90,1,3.85677
4,108,1,3.85677
5,114,1,3.85677
6,135,1,3.85677
7,152,1,3.85677
8,165,1,3.85677
9,182,1,3.85677


In [58]:
cv2.store_indices()

In [44]:
b = cv2.cross_validate_and_store(df, global_mean, 'global_mean')

/Users/joachimmuth/anaconda3/lib/python3.5/site-packages/pandas/types/dtypes.py:127: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


global_mean


In [63]:
dic = cv2.load_indices()

In [72]:
len(dic[1]['test'])

392317

In [50]:
b[2].to_csv('coucou.csv')

TypeError: 'NoneType' object is not subscriptable

In [ ]:
len(a[3]['train'])

In [ ]:
len(a[1]['test'])

In [ ]:
len(a[0]['test']) + len(a[0]['train'])

In [ ]:
len(a)

In [ ]:
cv.shuffle_indices(df, 3)

In [ ]:
cv.

In [ ]:
rescaler = Rescaler(df)
new_df = rescaler.normalize()

In [ ]:
new_df.head(5)

In [ ]:
train, test = split(df, 0.1)
train_norm, test_norm = split(new_df, 0.1)

In [ ]:
MF_SGD = matrix_factorization_SGD(train, test,n_iter=3)
print("MF_SGD: ", evaluate(MF_SGD, test))
user_m = user_mean(train, test)
print("user mean: ", evaluate(user_m, test))
movie_m = movie_mean(train, test)
print("movie_mean: ", evaluate(movie_m, test))

In [ ]:
blend = blender([MF_SGD, user_m, movie_m], [0.8, 0., 0.2])
print("blend: ", evaluate(blend, test))
blend.head()

## And now: rescaled

In [ ]:
MF_SGD_norm = rescaler.recover(matrix_factorization_SGD(train_norm, test,n_iter=3))
print("MF_SGD: ", evaluate(MF_SGD_norm, test))
user_m_norm = rescaler.recover(user_mean(train_norm, test))
print("user mean: ", evaluate(user_m_norm, test))
movie_m_norm = rescaler.recover(movie_mean(train_norm, test))
print("movie_mean: ", evaluate(movie_m_norm, test))

In [ ]:
blend_norm = blender([MF_SGD_norm, user_m_norm, movie_m_norm], [0.8, 0., 0.2])
print("blend: ", evaluate(blend_norm, test))
blend.head()

## Normalize only mean

In [ ]:
from rescaler import *
rescaler2 = Rescaler(df)
mean_df = rescaler2.normalize_only_mean()
train_norm_mean, test_norm_mean = split(mean_df, 0.1)

In [ ]:
MF_SGD_norm_mean.head(5)

In [ ]:
user_m_norm_mean = rescaler2.recover_only_mean(user_mean(train_norm_mean, test))
print("user mean: ", evaluate(user_m_norm_mean, test))
movie_m_norm_mean = rescaler2.recover_only_mean(movie_mean(train_norm_mean, test))
print("movie_mean: ", evaluate(movie_m_norm_mean, test))
MF_SGD_norm_mean = rescaler2.recover_only_mean(matrix_factorization_SGD(train_norm_mean, test,n_iter=3))
print("MF_SGD: ", evaluate(MF_SGD_norm_mean, test))

## deviation

In [ ]:
from rescaler import *
rescaler = Rescaler(df)
dev_df = rescaler.normalize_deviation()
train_dev, test_dev = split(dev_df, 0.1)

In [ ]:
user_m_dev = rescaler.recover_deviation(user_mean(train_dev, test))
print("user mean: ", evaluate(user_m_dev, test))
movie_m_dev = rescaler.recover_deviation(movie_mean(train_dev, test))
print("movie_mean: ", evaluate(movie_m_dev, test))
MF_SGD_dev = rescaler.recover_deviation(matrix_factorization_SGD(train_dev, test,n_iter=3))
print("MF_SGD: ", evaluate(MF_SGD_dev, test))

In [ ]:
print("MF_SGD standard: ", evaluate(MF_SGD, test))
print("MF_SGD deviation normalization: ", evaluate(MF_SGD_dev, test))
print("improve: ", evaluate(MF_SGD, test) - evaluate(MF_SGD_dev, test))

In [ ]:
d = dict(df.groupby('User').mean().Rating)

In [ ]:
np.mean(list(d.values()))

In [ ]:
sgd_ws = np.arange(0.1, 1, 0.02)
scores = []

for sgd_w in sgd_ws:
    mean_w = (1 - sgd_w) / 4
    blender = blend([MF_SGD, movie_m, user_m], [sgd_w, 4 * mean_w, 0* mean_w])
    score = evaluate(blender, test)
    scores.append(score)

In [ ]:
plt.plot(sgd_ws, scores)

In [ ]:
blender = blend2([MF_SGD, movie_m, user_m], [0.8, 0.2 , 0])

In [ ]:
blender.head()

## Do the prediction

In [ ]:
df_kaggle = load_csv_kaggle()

In [ ]:
df_kaggle.shape

In [ ]:
pred_MF = matrix_factorization_SGD(df, df_kaggle)

In [ ]:
pred_user = user_mean(df, df_kaggle)
pred_movie = movie_mean(df, df_kaggle)

In [ ]:
blend = blender([pred_MF, pred_movie, pred_user], [0.8, 0.2, 0])
blend.head()

In [ ]:
submission = submission_table(blend, 'User', 'Movie', 'Rating')

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../preds/blend_0.8MF_0.2MovieM.csv', index=False)